In [ ]:
# Instala a API do Google Earth Engine
#!pip install earthengine-api

import ee
ee.Authenticate()   # Autorize sua conta Google
ee.Initialize(project='luciano-ufsc')

In [ ]:
# Substitua pelos caminhos corretos dos seus assets
municipios = ee.FeatureCollection('projects/luciano-ufsc/assets/BR_Municipios_2024')
amostrasPoligono = ee.FeatureCollection('projects/luciano-ufsc/assets/AmostrasLEM2425V2')

In [ ]:
nomeMunicipio = 'Luís Eduardo Magalhães'

municipio = municipios.filter(
    ee.Filter.eq('NM_MUN', nomeMunicipio)
)

roi = municipio.geometry()

In [ ]:
def maskS2clouds(image):
    qa = image.select('QA60')

    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

In [ ]:
dataInicio = ee.Date('2025-01-01')
dataFim = ee.Date('2025-03-31')

s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(roi)
    .filterDate(dataInicio, dataFim)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(maskS2clouds)
    .select(['B2', 'B3', 'B4', 'B8', 'B11'])
)

In [ ]:
def addIndices(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')

    evi = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {
            'NIR': img.select('B8'),
            'RED': img.select('B4'),
            'BLUE': img.select('B2')
        }
    ).rename('EVI')

    savi = img.expression(
        '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
        {
            'NIR': img.select('B8'),
            'RED': img.select('B4')
        }
    ).rename('SAVI')

    gcvi = img.expression(
        '(NIR / GREEN) - 1',
        {
            'NIR': img.select('B8'),
            'GREEN': img.select('B3')
        }
    ).rename('GCVI')

    ndwi = img.normalizedDifference(['B3', 'B8']).rename('NDWI')

    sr = img.select('B8').divide(img.select('B4')).rename('SR')

    return img.addBands([ndvi, evi, savi, gcvi, ndwi, sr])

In [ ]:
withIndices = s2.map(addIndices)

In [ ]:
def addSuffix(img, suffix):
    bands = img.bandNames()
    renamed = bands.map(lambda b: ee.String(b).cat('_' + suffix))
    return img.rename(renamed)

mean = addSuffix(withIndices.mean(), 'mean')
median = addSuffix(withIndices.median(), 'median')
min_ = addSuffix(withIndices.min(), 'min')
max_ = addSuffix(withIndices.max(), 'max')

In [ ]:
image = (
    mean
    .addBands(median)
    .addBands(min_)
    .addBands(max_)
    .clip(roi)
)

In [ ]:
def polygonToPoints(feature):
    cultura = feature.get('Culturas')

    points = ee.FeatureCollection.randomPoints(
        region=feature.geometry(),
        points=950,
        seed=123
    ).map(lambda pt: pt.set('Culturas', cultura))

    return points

amostrasPontos = amostrasPoligono.map(polygonToPoints).flatten()

In [ ]:
sample = amostrasPontos.randomColumn('random')

trainingsample = sample.filter(ee.Filter.lte('random', 0.8))
validationsample = sample.filter(ee.Filter.gt('random', 0.8))

training = image.sampleRegions(
    collection=trainingsample,
    properties=['Culturas'],
    scale=10,
    tileScale=4
)

In [ ]:
RFclassifier = ee.Classifier.smileRandomForest(300).train(
    features=training,
    classProperty='Culturas',
    inputProperties=image.bandNames()
)

classified = image.classify(RFclassifier)

In [ ]:
nomeArquivo = (
    'RF_Classificacao_' +
    nomeMunicipio.replace(' ', '_').replace('í', 'i').replace('ã', 'a') + '_' +  # Corrected: Added .replace('ã', 'a')
    dataInicio.format('YYYY-MM-dd').getInfo() +
    '_a_' +
    dataFim.format('YYYY-MM-dd').getInfo()
)

task = ee.batch.Export.image.toDrive(
    image=classified.toInt8(),
    description=nomeArquivo,
    folder='GEE_RF',
    fileNamePrefix=nomeArquivo,
    region=roi,
    scale=10,
    maxPixels=1e13,
    fileFormat='GeoTIFF',
    formatOptions={'cloudOptimized': True}
)

task.start()